In [2]:
"""
Original logic generated by GPT-5.
Modified and verified by the researchers for the purposes of the proejct
Date: January 2026
Notes: Logic was adapted to create a graph with individual trajectories and debugged for reading the pickle data and clearing the pathes in update function.
"""
import os
import re
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

# configuration

DATA_DIR = "data"
OUTPUT_GIF = "results/erdos_renyi_lambda_sweep.gif"

beta = 1.5
FPS = 10  # frames per second
DPI = 300


# data extraction
def extract_lambda(filename: str) -> float:
    """
    Extract λ value from filename of the form:
    erdos_renyi_la_{lambda_value}.pkl
    """
    match = re.search(r"erdos_renyi_la_([0-9]+\.?[0-9]*(?:[eE][+-]?[0-9]+)?)", filename)
    if not match:
        raise ValueError(f"Could not extract λ from filename: {filename}")
    return float(match.group(1))


def load_runs(filepath: str) -> np.ndarray:
    """
    Load pickle file and return array of shape (n_runs, T).
    """
    with open(filepath, "rb") as f:
        data = pickle.load(f)

    data = np.asarray(data)
    data = np.mean(data, axis=2)

    return data

# data processing

files = sorted(
    [
        f for f in os.listdir(DATA_DIR)
        if f.startswith("erdos_renyi_") and f.endswith(".pkl")
    ],
    key=extract_lambda
)

lambdas = []
runs_per_lambda = []

for fname in files:
    l = extract_lambda(fname)
    runs = load_runs(os.path.join(DATA_DIR, fname))

    lambdas.append(l)
    runs_per_lambda.append(runs)


T = runs_per_lambda[0].shape[1]
time = np.arange(T)

# figure setup

fig, ax = plt.subplots(figsize=(9, 6))

mean_line, = ax.plot([], [], lw=2, label="Mean magnetization")
std_band = ax.fill_between([], [], [], alpha=0.3, label="±1σ")

ax.set_xlabel("Time steps")
ax.set_ylabel("Magnetization")
ax.set_ylim(-1, 1)
ax.grid(True, alpha=0.3)
ax.legend()

# Animation function
def update(frame_idx: int):
    for patch in ax.patches:
        patch.remove()  # remove previous std band

    runs = runs_per_lambda[frame_idx]
    l = lambdas[frame_idx]

    mean_m = runs.mean(axis=0)
    std_m = runs.std(axis=0)

    mean_line.set_data(time, mean_m)

    ax.fill_between(
        time,
        mean_m - std_m,
        mean_m + std_m,
        alpha=0.3
    )

    ax.set_title(
        f"Impact of Erdős–Rényi topology on belief alignment\n"
        f"Point: β = {beta}, λ = {l}"
    )

    ax.set_xlim(0, T - 1)

    return mean_line,

# render animation
anim = FuncAnimation(
    fig,
    update,
    frames=len(lambdas),
    interval=1000 // FPS,
    blit=False
)

writer = PillowWriter(fps=FPS)
anim.save(OUTPUT_GIF, writer=writer, dpi=DPI)

plt.close(fig)


### Second animation with individual trajectories

In [3]:
#adapted code to plot individual trajectories 

def update_individual(frame_idx: int):
    for line in ax.get_lines()[0:]:
        line.remove()  # remove previous liness

    runs = runs_per_lambda[frame_idx]
    l = lambdas[frame_idx]

    for run in runs:
        ax.plot(time, run, alpha = 0.8)

    ax.set_title(
        f"Impact of Erdős–Rényi topology on belief alignment\n"
        f"Point: β = {beta}, λ = {l}"
    )
    ax.set_xlim(0, T - 1)
    return ax.get_lines(),

# figure setup

fig, ax = plt.subplots(figsize=(9, 6))

ax.set_xlabel("Time steps")
ax.set_ylabel("Magnetization")
ax.set_ylim(-1, 1)
ax.grid(True, alpha=0.3)


animation_individual = FuncAnimation(
    fig,
    update_individual,
    frames=len(lambdas),
    interval=1000 // FPS,
    blit=False
)

writer = PillowWriter(fps=FPS)
animation_individual.save("results/erdos_renyi_individual.gif", writer=writer, dpi=DPI)

plt.close(fig)